---
### Objective of this notebook is to get a sample data set with the Google Users Reviews for GP clinics using Python & Google Maps API

1. For this sample use case I choose to query clinics related to the Raffles Clinics. 
2. To extract user reviews, we need to get the clinics details of the location.
3. Next, we need to find the place id of that location from place details (Step2).
4. Next, with place_id we can extract user reviews of this place. We can get only five reviews per place unless we own the place :)

---

In [ ]:
#!pip install -U googlemaps

In [58]:
import pandas as pd
import os

In [59]:
API_KEY = os.getenv('maps_api')
output_filename = 'data/raffles_clinics_test.csv'

### Get Google Reviews for Raffles Clinic

In [45]:
import googlemaps

In [60]:
gmaps = googlemaps.Client(key=API_KEY)

#### Step 1: Extract places IDs for Raffles clinics

We get 20 clinics (this is not a comprehencive list but based on the device location):

In [51]:
place_name = 'Raffles Clinic'
place_details = gmaps.places(place_name)
len(place_details['results'])

20

In [73]:
place_ids = [x['place_id'] for x in place_details['results']]
place_ids[0]

'ChIJbZHV_a892jERNhrm-bgNZXE'

#### Step 2: save only location and place id into a data frame:

In [68]:
df_location = pd.DataFrame(columns=['place_id', 'lat', 'lng'])

In [69]:
for i in range(0,len(place_details['results'])):
    df_location.loc[i] = [place_details['results'][i]['place_id'],
                          place_details['results'][i]['geometry']['location']['lat'],
                          place_details['results'][i]['geometry']['location']['lng'],
                         ]

In [70]:
df_location

,place_id,lat,lng
0,ChIJbZHV_a892jERNhrm-bgNZXE,1.372692,103.949590
1,ChIJY_X_PxI92jERr2tZ7YDdAx4,1.354276,103.945289
2,ChIJL7ZNaFQ82jERG0nE6vQksRM,1.366480,103.964491
3,ChIJNbUBTyI92jERJ9Cx-rO_VPU,1.343062,103.953104
4,ChIJDQ5i_q892jEREshja_0JOeU,1.372618,103.949543
5,ChIJaTRtzNk82jERlu6VPkoABQk,1.335020,103.962500
6,ChIJXTiNyrIi2jER1vY4SDJ1gCo,1.325633,103.929022
7,ChIJn_mqNwUW2jER8HTHiDGKPYs,1.392256,103.904421
8,ChIJn7DHqDcW2jERMYgjyehk7U8,1.371255,103.893633
9,ChIJ_-4P-ow82jERuJBbBoncXa0,1.356716,103.986510


#### Step 3: Create a data frame to save top 5 reviews by place is

In [87]:
df_review = pd.DataFrame(columns=['place_id', 'reviews'])

In [ ]:
index = 0
for i in range(0,len(place_ids)):
    print(i)
    place_i = gmaps.place(place_ids[i])
    if "reviews" in place_i['result'].keys():
        for j in range(0,len(place_i['result']['reviews'])):
                   df_review.loc[index] = [place_ids[i],place_i['result']['reviews'][j]['text']]
                   index=index+1

               

In [89]:
df_review.head()

,place_id,reviews
0,ChIJbZHV_a892jERNhrm-bgNZXE,The staff is very kind and patient. Unfortunat...
1,ChIJbZHV_a892jERNhrm-bgNZXE,"Tested positive for dengue, and the first two ..."
2,ChIJbZHV_a892jERNhrm-bgNZXE,I wasn't very impressed with the doctor during...
3,ChIJbZHV_a892jERNhrm-bgNZXE,I called to only ask if I can get a medicine b...
4,ChIJbZHV_a892jERNhrm-bgNZXE,I wanted to scan for registration but the staf...


#### Step 4: Save Results into one dataframe

In [90]:
df = pd.merge(df_review, df_location, on='place_id', how='left')

In [91]:
df.shape

(87, 4)

In [92]:
df.head()

,place_id,reviews,lat,lng
0,ChIJbZHV_a892jERNhrm-bgNZXE,The staff is very kind and patient. Unfortunat...,1.372692,103.94959
1,ChIJbZHV_a892jERNhrm-bgNZXE,"Tested positive for dengue, and the first two ...",1.372692,103.94959
2,ChIJbZHV_a892jERNhrm-bgNZXE,I wasn't very impressed with the doctor during...,1.372692,103.94959
3,ChIJbZHV_a892jERNhrm-bgNZXE,I called to only ask if I can get a medicine b...,1.372692,103.94959
4,ChIJbZHV_a892jERNhrm-bgNZXE,I wanted to scan for registration but the staf...,1.372692,103.94959


##### Save Results to csv.

In [93]:
df.to_csv('data/raffles_top5_reviews.csv', index=False)

#### Next Step we will analyse the reviews with OpenAI api